# Prototyping fallback gen & use script in functions_fallback.R

In [1]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [2]:
# install.packages("pkgcond")

In [3]:
library(dplyr)
library(data.table)
library(partykit)
library(tictoc)
library(caret)
library(e1071)
library(randomForest)
library(ranger)

#for 3d plotting
library(akima)
library(plotly)

# for prep data
# library(rPython)
library(stringr)
library(pbapply)
library(stringdist)
library(data.table)
library(dominanceanalysis)
library(zoo)


getwd()


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: grid

Loading required package: libcoin

Loading required package: mvtnorm

Loading required package: lattice

Loading required package: ggplot2

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘ranger’


The following object is masked from ‘package:randomForest’:

    importance



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:sta

[1] "/home/jupyter/Domains_202003/scripts/phaseII_06_fallbacksupp"

In [4]:
# install.packages(c("zoo"))

In [5]:

source('../orig/functions.R')

source('../orig/functions_models.R')

source('../phaseII_03_forest/functions_eval.R')



Attaching package: ‘tidyr’


The following object is masked from ‘package:stringdist’:

    extract




In [6]:
source('functions_metalearning.R')

In [7]:
source('functions_fallback.R')

# Load old & New expiry data

In [8]:
# 5 extra observations... perhaps due to new flags?
expiry_df_old <- readRDS("/home/jupyter/Domains_202003/data/output/expiry_20190601_20200901")
expiry_df_new <- readRDS("/home/jupyter/Domains_202003/data/output/expiry_20190601_20200901_20201116")

dim(expiry_df_old)
dim(expiry_df_new)

[1] 5427633      29

[1] 5427638      30

In [9]:
# after preprocessing data...
expiry_df_old2 <- expiry_df_old %>% 
   filter(expiry_date >= as.Date("2019-06-01") & expiry_date <= as.Date("2020-09-01"))  %>% 
   filter(renewed_count==1) %>% 
   filter(!is.na(gibb_score))

expiry_df_new2 <- expiry_df_new %>% 
   filter(expiry_date >= as.Date("2019-06-01") & expiry_date <= as.Date("2020-09-01"))  %>% 
   filter(renewed_count==1) %>% 
   filter(!is.na(gibb_score))


dim(expiry_df_old2)
dim(expiry_df_new2)

[1] 4661731      29

[1] 4661731      30

In [10]:
domain_id_old2 <- expiry_df_old2 %>% distinct(domain_id) %>% pull(domain_id)
domain_id_new2 <- expiry_df_new2 %>% distinct(domain_id) %>% pull(domain_id)

length(domain_id_old2)
length(domain_id_new2)

[1] 4661731

[1] 4661731

In [11]:
setdiff(domain_id_new2,domain_id_old2)
setdiff(domain_id_old2,domain_id_new2)

integer(0)

integer(0)

# ADD vars to expiry_df_new2

In [13]:
expiry_df_new2 <- expiry_df_new2 %>% mutate (reg_arpt = ifelse(reg_arpt <= 0, 0.0001,reg_arpt),
                                   log_reg_arpt = log(reg_arpt),
                                   tld_registrar_index = tolower(paste(tld, reseller,sep="")))

# CREATE new train & test & preds dataframes <br>based on new data w/ region variable

In [12]:
# load old preds
preds_df1 <- read.csv('../../data/output/models_20201104/preds.csv')
preds_df2 <- read.csv('../../data/output/models_20201104/preds/preds.csv')
preds_df <- rbind(preds_df1,preds_df2)
dim(preds_df)

[1] 932347     39

In [18]:
# no missing preds for agg_rf_ALL so use that to identify test data
preds_df %>% filter(is.na(pred_agg_rf_ALL))

renewal_type,renewed_count,expiry_date,creation_date,status,domain_id,domain,registrar,reseller,reseller_country,registrant_country,reg_period,reg_revenue,reg_arpt,renew_mbg,renewal_item_id,renew_type,autorenew_type,renewal_status,renew_date,renew_arpt,gibb_score,pattern,pattern_domain_count,day_domains,sld_length,sld_type,sld_type2,tld,log_reg_arpt,tld_registrar_index,pred_agg_glm_ALL,pred_agg_rf_ALL,pred_agg_glm,pred_agg_rf,pred_seg_glm_ALL,pred_seg_rf_ALL,pred_seg2_glm_ALL,pred_seg2_rf_ALL
<fct>,<int>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<dbl>,<dbl>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


In [16]:
expiry_df_new3 <- merge(expiry_df_new2, 
                        preds_df %>% select(domain_id, contains('pred_')),
                       by='domain_id', all.x=TRUE)
dim(expiry_df_new2)
dim(expiry_df_new3)

[1] 4661731      32

[1] 4661731      40

In [21]:
head(expiry_df_new3)

,domain_id,renewal_type,renewed_count,expiry_date,creation_date,status,domain,registrar,reseller,reseller_country,registrant_country,reg_period,reg_revenue,reg_arpt,renew_mbg,renewal_item_id,renew_type,autorenew_type,renewal_status,renew_date,renew_arpt,gibb_score,pattern,pattern_domain_count,day_domains,sld_length,sld_type,sld_type2,tld,region,log_reg_arpt,tld_registrar_index,pred_agg_glm_ALL,pred_agg_rf_ALL,pred_agg_glm,pred_agg_rf,pred_seg_glm_ALL,pred_seg_rf_ALL,pred_seg2_glm_ALL,pred_seg2_rf_ALL
,<int>,<chr>,<int>,<date>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,41612421,FirstTime,1,2020-01-01,2017-01-01,Active,bluebyebye.store,Go Daddy,Go Daddy,United States,China,3,83.00,27.67,0,273918686,auto-renewal,unrealized,Not Renewd,2020-01-01,40,2.72,bluebyebye,1,16,10,l,10l,store,Non China,3.32034879,storego daddy,NA,NA,NA,NA,NA,NA,NA,NA
2,41613156,FirstTime,1,2020-01-01,2017-01-01,Active,rxmjqj.pw,Shanghai Meicheng,Shanghai Meicheng,China,China,3,3.25,1.08,0,273918431,auto-renewal,unrealized,Not Renewd,2020-01-01,3,0.04,rxmjqj,1,42,6,l,6l,pw,China,0.07696104,pwshanghai meicheng,NA,NA,NA,NA,NA,NA,NA,NA
3,41613923,FirstTime,1,2020-01-01,2017-01-01,Active,wastecase.space,PDR Ltd,Domain.com,United States,Canada,3,13.00,4.33,0,269044961,renewal,NA,Renewed,2019-12-21,6,9.65,wastecase,1,3,9,l,9l,space,Non China,1.46556754,spacedomain.com,NA,NA,NA,NA,NA,NA,NA,NA
4,41613952,FirstTime,1,2020-01-01,2017-01-01,Active,jamieandandrew.space,Go Daddy,Go Daddy,United States,China,3,12.50,4.17,0,273918906,auto-renewal,unrealized,Not Renewd,2020-01-01,15,6.88,jamieandandrew,1,14,14,l,14l,space,Non China,1.42791604,spacego daddy,NA,NA,NA,NA,NA,NA,NA,NA
5,41616161,FirstTime,1,2020-01-01,2017-01-01,Active,zangshimei.space,Shanghai Meicheng,Shanghai Meicheng,China,China,3,3.50,1.17,0,273918441,auto-renewal,unrealized,Not Renewd,2020-01-01,5,6.80,zangshimei,1,1,10,l,10l,space,China,0.15700375,spaceshanghai meicheng,NA,NA,NA,NA,NA,NA,NA,NA
6,41617303,FirstTime,1,2020-01-01,2017-01-01,Active,janezdm.space,Xin Net,Xin Net,China,China,3,3.50,1.17,0,273893916,auto-renewal,unrealized,Not Renewd,2020-01-01,5,1.72,janezdm,1,3,7,l,7l,space,China,0.15700375,spacexin net,0.12,0.1733994,0.115,0.1052066,0.105,0.1503908,0.08616119,0.2433018


In [24]:
expiry_df_train <- expiry_df_new3 %>% filter(is.na(pred_agg_rf_ALL))
expiry_df_test_preds <- expiry_df_new3 %>% filter(!is.na(pred_agg_rf_ALL))
expiry_df_test <- expiry_df_test_preds %>% select(-contains('pred_'))

In [25]:
dim(expiry_df_train)/dim(expiry_df_new2)
dim(expiry_df_test)/dim(expiry_df_new2)
dim(expiry_df_test_preds)
dim(preds_df)

[1] 0.7999998 1.2500000

[1] 0.2000002 1.0000000

[1] 932347     40

[1] 932347     39

In [26]:
# WRITE to CSV's
write.csv(expiry_df_test_preds, "../../data/output/datapull_20201116/expiry_df_test_preds.csv",
         row.names=FALSE)
write.csv(expiry_df_train, "../../data/output/datapull_20201116/expiry_df_train.csv",
         row.names=FALSE)
write.csv(expiry_df_test, "../../data/output/datapull_20201116/expiry_df_test.csv",
         row.names=FALSE)


In [ ]:
# WRITE to GCP shared output

system('gsutil cp -r /home/jupyter/Domains_202003/data/output/datapull_20201116 gs://output_shared/')